# Day 4, Session 2: LSTM Drought Monitoring Lab - INSTRUCTOR VERSION
## Complete Solution for Mindanao Drought Forecasting

This notebook contains complete working code for the LSTM drought monitoring lab.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Machine Learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

plt.rcParams['figure.figsize'] = (14, 6)
sns.set_style('whitegrid')

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## Generate Data

In [ ]:
def generate_mindanao_drought_data(start_date='2015-01-01', end_date='2021-12-31'):
    dates = pd.date_range(start=start_date, end=end_date, freq='MS')
    n_months = len(dates)
    
    ndvi = np.zeros(n_months)
    rainfall = np.zeros(n_months)
    temperature = np.zeros(n_months)
    oni = np.zeros(n_months)
    
    base_ndvi = 0.70
    
    for i, date in enumerate(dates):
        month = date.month
        year = date.year
        
        if month in [11, 12, 1, 2, 3]:
            seasonal_factor = 0.85 + 0.10 * np.sin(2 * np.pi * month / 12)
            rain_base = 250
            temp_base = 26
        else:
            seasonal_factor = 0.75 + 0.05 * np.sin(2 * np.pi * month / 12)
            rain_base = 80
            temp_base = 28
        
        drought_factor = 1.0
        if year == 2015 and month >= 6:
            drought_factor = 0.60
            oni[i] = 2.5 + np.random.randn() * 0.3
            rain_base *= 0.4
            temp_base += 2
        elif year == 2016 and month <= 6:
            drought_factor = 0.65
            oni[i] = 2.0 + np.random.randn() * 0.3
            rain_base *= 0.5
            temp_base += 1.5
        else:
            oni[i] = np.random.randn() * 0.5
        
        ndvi[i] = base_ndvi * seasonal_factor * drought_factor + np.random.normal(0, 0.03)
        ndvi[i] = np.clip(ndvi[i], 0.2, 0.9)
        rainfall[i] = max(0, rain_base + np.random.normal(0, 40))
        temperature[i] = temp_base + np.random.normal(0, 1.5)
    
    df = pd.DataFrame({
        'date': dates,
        'ndvi': ndvi,
        'rainfall': rainfall,
        'temperature': temperature,
        'oni': oni
    })
    
    return df

df = generate_mindanao_drought_data()
print(f"Generated {len(df)} months")
print(df.head())

## Visualization

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(15, 10), sharex=True)

axes[0].plot(df['date'], df['ndvi'], 'g-', linewidth=2)
axes[0].axvspan(pd.Timestamp('2015-06-01'), pd.Timestamp('2016-06-01'), 
                alpha=0.2, color='red', label='2015-16 Drought')
axes[0].set_ylabel('NDVI')
axes[0].set_title('Mindanao NDVI (2015-2021)', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].bar(df['date'], df['rainfall'], color='blue', alpha=0.6, width=20)
axes[1].set_ylabel('Rainfall (mm)')
axes[1].grid(True, alpha=0.3)

colors = ['red' if x > 0.5 else 'blue' if x < -0.5 else 'gray' for x in df['oni']]
axes[2].bar(df['date'], df['oni'], color=colors, alpha=0.7, width=20)
axes[2].axhline(y=0.5, color='red', linestyle='--', alpha=0.5)
axes[2].set_ylabel('ONI Index')
axes[2].set_xlabel('Date')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Exercise 1 Solution

In [ ]:
df['month'] = df['date'].dt.month

# Seasonal means
dry_season_ndvi = df[df['month'].isin([5,6,7,8,9,10])]['ndvi'].mean()
wet_season_ndvi = df[df['month'].isin([11,12,1,2,3,4])]['ndvi'].mean()

print(f"Dry season mean NDVI: {dry_season_ndvi:.3f}")
print(f"Wet season mean NDVI: {wet_season_ndvi:.3f}")

# Lowest NDVI
lowest_idx = df['ndvi'].idxmin()
print(f"\nLowest NDVI: {df.loc[lowest_idx, 'ndvi']:.3f} on {df.loc[lowest_idx, 'date']}")

# Correlation
correlation = df['ndvi'].corr(df['rainfall'])
print(f"\nNDVI-Rainfall correlation: {correlation:.3f}")

## Sequence Creation

In [ ]:
LOOKBACK_WINDOW = 12
FORECAST_HORIZON = 1

feature_columns = ['ndvi', 'rainfall', 'temperature', 'oni']
target_column = 'ndvi'

scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = df.copy()
df_scaled[feature_columns] = scaler.fit_transform(df[feature_columns])

def create_sequences(data, features, target, lookback, horizon):
    X, y, dates = [], [], []
    feature_data = data[features].values
    target_data = data[target].values
    date_data = data['date'].values
    
    for i in range(lookback, len(data) - horizon + 1):
        X.append(feature_data[i - lookback:i])
        y.append(target_data[i + horizon - 1])
        dates.append(date_data[i + horizon - 1])
    
    return np.array(X), np.array(y), np.array(dates)

X, y, dates = create_sequences(df_scaled, feature_columns, target_column, 
                                LOOKBACK_WINDOW, FORECAST_HORIZON)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

## Train-Val-Test Split

In [ ]:
train_end = pd.Timestamp('2019-12-31')
val_end = pd.Timestamp('2020-12-31')

train_mask = dates <= train_end
val_mask = (dates > train_end) & (dates <= val_end)
test_mask = dates > val_end

X_train, y_train = X[train_mask], y[train_mask]
X_val, y_val = X[val_mask], y[val_mask]
X_test, y_test = X[test_mask], y[test_mask]

dates_train = dates[train_mask]
dates_val = dates[val_mask]
dates_test = dates[test_mask]

print(f"Train: {len(X_train)} sequences")
print(f"Val: {len(X_val)} sequences")
print(f"Test: {len(X_test)} sequences")

## Build LSTM Model

In [ ]:
def build_lstm_model(input_shape, lstm_units=[64, 32], dropout=0.2, learning_rate=0.001):
    model = Sequential(name='LSTM_Drought_Forecaster')
    
    # First LSTM layer
    model.add(LSTM(lstm_units[0], return_sequences=True, input_shape=input_shape, name='LSTM_1'))
    model.add(Dropout(dropout, name='Dropout_1'))
    
    # Second LSTM layer
    model.add(LSTM(lstm_units[1], return_sequences=False, name='LSTM_2'))
    model.add(Dropout(dropout, name='Dropout_2'))
    
    # Dense layers
    model.add(Dense(16, activation='relu', name='Dense_1'))
    model.add(Dense(1, activation='linear', name='Output'))
    
    # Compile
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae', 'mse'])
    
    return model

input_shape = (LOOKBACK_WINDOW, len(feature_columns))
model = build_lstm_model(input_shape)
model.summary()

## Train Model

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6, verbose=1)
]

BATCH_SIZE = 16
EPOCHS = 100

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

print("\nTraining complete!")

## Training History

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(history.history['loss'], label='Train Loss')
axes[0].plot(history.history['val_loss'], label='Val Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('MSE Loss')
axes[0].set_title('Training History', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(history.history['mae'], label='Train MAE')
axes[1].plot(history.history['val_mae'], label='Val MAE')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].set_title('MAE History', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Evaluation

In [ ]:
y_test_pred = model.predict(X_test).flatten()

def inverse_transform_ndvi(values, scaler, feature_columns):
    dummy = np.zeros((len(values), len(feature_columns)))
    ndvi_idx = feature_columns.index('ndvi')
    dummy[:, ndvi_idx] = values
    inverse = scaler.inverse_transform(dummy)
    return inverse[:, ndvi_idx]

y_test_actual = inverse_transform_ndvi(y_test, scaler, feature_columns)
y_test_pred_original = inverse_transform_ndvi(y_test_pred, scaler, feature_columns)

rmse = np.sqrt(mean_squared_error(y_test_actual, y_test_pred_original))
mae = mean_absolute_error(y_test_actual, y_test_pred_original)
r2 = r2_score(y_test_actual, y_test_pred_original)

print(f"\nTest Set Performance:")
print(f"  RMSE: {rmse:.4f}")
print(f"  MAE:  {mae:.4f}")
print(f"  R²:   {r2:.4f}")

## Visualize Predictions

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Time series
axes[0].plot(dates_test, y_test_actual, 'g-', linewidth=2, marker='o', 
             markersize=4, label='Actual NDVI')
axes[0].plot(dates_test, y_test_pred_original, 'r--', linewidth=2, marker='x', 
             markersize=4, label='Predicted NDVI')
axes[0].set_xlabel('Date', fontsize=12)
axes[0].set_ylabel('NDVI', fontsize=12)
axes[0].set_title('LSTM Drought Forecasting: Test Set (2021)', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Scatter plot
axes[1].scatter(y_test_actual, y_test_pred_original, alpha=0.6, edgecolors='k', linewidths=0.5)
min_val = min(y_test_actual.min(), y_test_pred_original.min())
max_val = max(y_test_actual.max(), y_test_pred_original.max())
axes[1].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect')
axes[1].set_xlabel('Actual NDVI', fontsize=12)
axes[1].set_ylabel('Predicted NDVI', fontsize=12)
axes[1].set_title('Prediction Accuracy', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Operational Analysis

In [ ]:
DROUGHT_THRESHOLD = 0.4

predicted_drought = y_test_pred_original < DROUGHT_THRESHOLD
actual_drought = y_test_actual < DROUGHT_THRESHOLD

true_positives = np.sum(predicted_drought & actual_drought)
false_positives = np.sum(predicted_drought & ~actual_drought)
true_negatives = np.sum(~predicted_drought & ~actual_drought)
false_negatives = np.sum(~predicted_drought & actual_drought)

precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
false_alarm_rate = false_positives / (false_positives + true_negatives) if (false_positives + true_negatives) > 0 else 0

print("\n📊 Operational Performance (NDVI < 0.4 = Drought):")
print(f"  True Positives: {true_positives}")
print(f"  False Positives: {false_positives}")
print(f"  True Negatives: {true_negatives}")
print(f"  False Negatives: {false_negatives}")
print(f"\n  Precision: {precision:.2%}")
print(f"  Recall: {recall:.2%}")
print(f"  False Alarm Rate: {false_alarm_rate:.2%}")

print("\n✅ Model is ready for operational deployment!")
print("   Integration points:")
print("   - PAGASA seasonal forecast system")
print("   - DA agricultural advisory platform")
print("   - PhilSA Space+ Dashboard")